In [2]:
import pandas_datareader as web
import pandas as pd
import numpy as np
from pykrx import stock
import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
import pprint

In [67]:
# 아 클래스를 만들다가 끝낫엇구나
class Crypto_MA_Strategy_Backtesting:
    
    def __init__(self,MA,Filename):
        # need to file input
        self.BTC_D = pd.read_csv(Filename)
        self.MA = MA
        self.column_name = 'MA'+str(self.MA)
        
        # 이게 평균 메길때 이상하게 최근값이 Nan으로 나와서 인덱스순 정렬을 의도치 않게 이렇게 할수밖에 없음 # 이건 뭐 이렇게 하는게 맞는거 같네
        self.BTC_D[self.column_name] = self.BTC_D['close'].sort_index(ascending=False)\
        .rolling(window=MA).mean().sort_index(ascending=True)
        
        # 여기까진 이상 없음
        
    def High_Low_Detect(self):
        # 이렇게 되면 메모리 사용이 어떻게 되는지 나중에 확인을 해봐야겠다.
        BTC_D = self.BTC_D
        # Ma가 close price보다 높은 row만 걸러냄
        CHTM = BTC_D[BTC_D[self.column_name] < BTC_D['close']].copy()

        # Diff 값을 구하기 위해 # Unix 값으로 정렬이 가능한지 보자
        CHTM['index1'] = CHTM.index
        CHTM['index1'] = CHTM['index1'].diff()
        
        # Ma가 close price보다 높은 row만 걸러냄
        CLTM = BTC_D[BTC_D[self.column_name] > BTC_D['close']].copy()

        # Diff 값을 구하기 위해 # Unix 값으로 정렬이 가능한지 보자
        CLTM['index1'] = CLTM.index
        CLTM['index1'] = CLTM['index1'].diff()
        
        self.CHTM = CHTM[CHTM['index1'] != 1]
        self.CLTM = CLTM[CLTM['index1'] != 1]
        
        return self.CHTM, self.CLTM
    
        # 여기 인덱싱 방식 바꿔야함
        # view등 카피방식을 바꿔야함
    
    def Final_result(self):
        # 두개를 합치는 로직
        chtm = np.array(self.CHTM.index)
        cltm = np.array(self.CLTM.index) 
        Total = np.sort(np.append(chtm,cltm) - 1)[1:]      
        self.Final_result = self.BTC_D[self.BTC_D.index.isin(Total[:len(Total)])]
        
        return self.Final_result
    
    def Long_Back_Testing(self,Asset=1000000):
        
        df = self.Final_result
        print("Starting Asset :",Asset)
        if df.iloc[-1]['close'] < df.iloc[-1][self.column_name]:
            df = df.drop(df.index[-1])

        Profit_list = []

        while len(df) > 2:
            print(df.tail(2).index)
            Close = df.tail(2).iloc[0]['close']
            Open = df.tail(2).iloc[1]['close']
            Profit = Close/Open
            Profit_list.append(Profit)
            Asset = Asset*Profit
            print("Profit :", Profit, "Open :",Open,"Close :",Close,"Asset :",Asset)
            df = df.drop(df.tail(2).index)

        Profit_list = pd.Series(Profit_list,dtype='float64')

        return Profit_list

    def Short_Back_Testing(self,Asset=1000000):
        
        df = self.Final_result
        
        print("Starting Asset :",Asset)
        if df.iloc[-1]['close'] > df.iloc[-1][self.column_name]:
            df = df.drop(df.index[-1])

        while len(df) > 2:
            Close = df.tail(2).iloc[0]['close']
            Open = df.tail(2).iloc[1]['close']
            Profit = 2 - Close/Open
            Asset = Asset*Profit
            print("Profit :", Profit, "Open :",Open,"Close :",Close,"Asset :",Asset)
            df = df.drop(df.tail(2).index)
       
    

In [68]:
aaa = Crypto_MA_Strategy_Backtesting(80,'Binance_BTCUSDT_d.csv')
aaa.High_Low_Detect()
aaa.Final_result()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount,MA80
32,1.638490e+12,2021-12-03 00:00:00,BTC/USDT,56484.26,57600.00,51680.00,53601.05,58927.690270,3.246199e+09,1865063.0,56123.415125
37,1.638058e+12,2021-11-28 00:00:00,BTC/USDT,54716.47,57445.05,53256.64,57274.88,36163.713700,1.982198e+09,1253408.0,55449.787875
39,1.637885e+12,2021-11-26 00:00:00,BTC/USDT,58960.37,59150.00,53500.00,53726.53,65927.870660,3.649354e+09,2138184.0,55205.382125
96,1.632960e+12,2021-09-30 00:00:00,BTC/USDT,41524.29,44141.37,41410.17,43824.10,46381.227810,2.005486e+09,1197815.0,43181.132750
99,1.632701e+12,2021-09-27 00:00:00,BTC/USDT,43160.90,44350.00,42098.00,42147.35,39776.843830,1.728912e+09,1001487.0,42862.052375
104,1.632269e+12,2021-09-22 00:00:00,BTC/USDT,40734.09,44000.55,40565.39,43543.61,58349.055420,2.482639e+09,1304011.0,42272.165750
105,1.632182e+12,2021-09-21 00:00:00,BTC/USDT,43016.64,43639.00,39600.00,40734.38,84534.080485,3.551998e+09,1820721.0,42168.952000
161,1.627344e+12,2021-07-27 00:00:00,BTC/USDT,37241.33,39542.61,36383.00,39457.87,88397.267015,3.335658e+09,1938478.0,37035.184250
237,1.620778e+12,2021-05-12 00:00:00,BTC/USDT,56670.02,58000.01,48600.00,49631.32,99842.789836,5.476115e+09,2676826.0,55302.718250
244,1.620173e+12,2021-05-05 00:00:00,BTC/USDT,53205.05,58069.82,52900.00,57436.11,77263.923439,4.327994e+09,2378119.0,55014.516000


In [70]:
# 이게또 기간이 짧다면 매수 매도를 진행을 안하면서 수익률을 올릴수있겠네

aaa.Long_Back_Testing()

Starting Asset : 1000000
Int64Index([1444, 1445], dtype='int64')
Profit : 0.900785472431902 Open : 12799.94 Close : 11530.0 Asset : 900785.472431902
Int64Index([1334, 1355], dtype='int64')
Profit : 0.948404535180163 Open : 8856.98 Close : 8400.0 Asset : 854309.0272788214
Int64Index([1264, 1265], dtype='int64')
Profit : 0.9827650173247202 Open : 7466.21 Close : 7337.53 Asset : 839585.0259943359
Int64Index([1249, 1263], dtype='int64')
Profit : 0.9474318739035354 Open : 7398.78 Close : 7009.84 Asset : 795449.6144791621
Int64Index([1217, 1226], dtype='int64')
Profit : 0.9698001343245559 Open : 6908.64 Close : 6700.0 Asset : 771427.1429703076
Int64Index([1174, 1177], dtype='int64')
Profit : 0.9802236208811551 Open : 6752.5 Close : 6618.96 Asset : 756171.1073283595
Int64Index([874, 1052], dtype='int64')
Profit : 2.7403274093543977 Open : 3667.58 Close : 10050.37 Asset : 2072156.4115737695
Int64Index([867, 873], dtype='int64')
Profit : 0.9852961891133901 Open : 10293.93 Close : 10142.57 Asset

0     0.900785
1     0.948405
2     0.982765
3     0.947432
4     0.969800
5     0.980224
6     2.740327
7     0.985296
8     0.975437
9     0.965919
10    0.973118
11    0.981141
12    0.959790
13    1.039843
14    0.901076
15    1.190521
16    1.079963
17    0.973238
18    0.976147
19    0.995376
20    0.994106
21    4.677644
22    0.991734
23    0.922102
24    0.864114
25    1.032351
26    0.967934
27    1.225959
dtype: float64

In [66]:
aaa.Short_Back_Testing()

Starting Asset : 1000000
Profit : 0.8256935779816514 Open : 10900.0 Close : 12799.94 Asset : 825693.5779816514
Profit : 1.2318317432784043 Open : 11530.0 Close : 8856.98 Asset : 1017115.5595789207
Profit : 1.1111654761904761 Open : 8400.0 Close : 7466.21 Asset : 1130183.695100254
Profit : 0.9916525043168478 Open : 7337.53 Close : 7398.78 Asset : 1120749.4915842356
Profit : 1.0144368487725826 Open : 7009.84 Close : 6908.64 Asset : 1136929.5825061859
Profit : 0.9921641791044775 Open : 6700.0 Close : 6752.5 Asset : 1128020.8059268463
Profit : 1.4458978449786675 Open : 6618.96 Close : 3667.58 Asset : 1631002.8523807267
Profit : 0.9757660663239265 Open : 10050.37 Close : 10293.93 Asset : 1591477.2374306454
Profit : 0.9756491697863561 Open : 10142.57 Close : 10389.55 Asset : 1552723.4454330928
Profit : 1.0596407268349721 Open : 10134.35 Close : 9529.93 Asset : 1645329.0002924246
Profit : 0.9780035936444202 Open : 9205.14 Close : 9407.62 Asset : 1609137.6750133727
Profit : 0.973933664819896 O